# **[Link to Kaggle™ Dataset](https://www.kaggle.com/datasets/puneet6060/intel-image-classification/)**

**Mounting Google Drive to access local dataset files.**

---



In [1]:
from google.colab import drive
drive.mount('/content/drive')
# /content/drive/MyDrive/insert path

Mounted at /content/drive


**Importing all the necessary libraries and modules**

---



In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils import to_categorical, image_dataset_from_directory
import matplotlib.pyplot as plt
from keras.applications import Xception, EfficientNetB2
import os
import numpy as np
from PIL import Image
from keras.preprocessing.image  import ImageDataGenerator    # Keras libaray to manage image files

**Defining functions that will allow us to load the individual images from the dataset into NumPy arrays useful for feeding a model...**

---



In [3]:
# Alternative and effective code for loading image data

In [4]:
base_dir = '/content/drive/Othercomputers/My Surface Laptop/1hundred percent Coding/Personal Programming/ML/Scenes of Nature Dataset'
train_dir = os.path.join(base_dir, 'seg_train/seg_train')
validation_dir = os.path.join(base_dir,'seg_test/seg_test')
test_dir = os.path.join(base_dir, 'seg_pred')

In [5]:
train_buildings_dir = os.path.join(train_dir, 'buildings')
val_buildings_dir = os.path.join(validation_dir, 'buildings')
test_buildings_dir = os.path.join(test_dir, 'buildings')

train_forest_dir = os.path.join(train_dir, 'forest')
val_forest_dir = os.path.join(validation_dir, 'forest')
test_forest_dir = os.path.join(test_dir, 'forest')

train_glacier_dir = os.path.join(train_dir, 'glacier')
val_glacier_dir = os.path.join(validation_dir, 'glacier')
test_glacier_dir = os.path.join(test_dir, 'glacier')

train_mountain_dir = os.path.join(train_dir, 'mountain')
val_mountain_dir = os.path.join(validation_dir, 'mountain')
test_mountain_dir = os.path.join(test_dir, 'mountain')

train_sea_dir = os.path.join(train_dir, 'sea')
val_sea_dir = os.path.join(validation_dir, 'sea')
test_sea_dir = os.path.join(test_dir, 'sea')

train_street_dir = os.path.join(train_dir, 'street')
val_street_dir = os.path.join(validation_dir, 'street')
test_street_dir = os.path.join(test_dir, 'street')

In [6]:
# train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
   train_dir,                                           # target directory
   target_size=(150, 150),                  # resize all images to 150 x 150
   batch_size=100,                                # batch size
   class_mode='categorical'
)

Found 6000 images belonging to 6 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(    # repeat the same with the validation 							       #samples
  validation_dir,
  target_size=(150, 150),
	batch_size=20,
  class_mode='categorical'
)

Found 3000 images belonging to 6 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(    # repeat the same with the validation 							       #samples
  test_dir,
  target_size=(150, 150),
	batch_size=20,
  class_mode='categorical'
)

Found 3000 images belonging to 6 classes.


**Calling the Data Loader functions defined earlier**

---



**Creating the Organic model and adding the necessary layers**

---



**Compiling the model with the apporopriate loss function and an optimizer then fitting it to the Scenes from Nature dataset**

---



In [ ]:
conv_base = Xception(weights='imagenet',	include_top=False, input_shape=(150, 150, 3))

83683744/83683744 [==============================] - 0s 0us/step


In [ ]:
def extract_features(generator, sample_count):
    batch_size = generator.batch_size
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    print('still alive')
    i = 0
    for inputs_batch, labels_batch in generator:
        i = generator.index(inputs_batch)
        print(f'still alive {i}')
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
#         i += 1
        if i * batch_size >= sample_count:
            print(f'still alive {i}')
            break
    return features, labels
#     for inputs_batch, labels_batch in generator:
#         i = generator.index(inputs_batch)
#         print(f'still alive {i}')
#         features_batch = conv_base.predict(inputs_batch)
#         features[i * batch_size : (i + 1) * batch_size] = features_batch
#         labels[i * batch_size : (i + 1) * batch_size] = labels_batch
#         i += 1
#         if i * batch_size >= sample_count:
#             print(f'still alive {i}')
#             break
#     return features, labels

In [ ]:
features_training, labels_training = extract_features(train_generator, 6000)
features_validating, labels_validating = extract_features(validation_generator, 3000)
features_testing, labels_testing = extract_features(test_generator, 3000)

In [ ]:
features_training = np.reshape(features_training, (6000, 5 * 5 * 2048))
features_validating = np.reshape(features_validating, (3000, 5 * 5 * 2048))
features_testing = np.reshape(features_testing, (3000, 5 * 5 * 2048))